#Setup google-drive mounting (optional)

In [ ]:
# from google.colab import drive
# import os

# drive.mount('/content/drive')

In [ ]:
# # prompt: create a symbolic link to a google drive workdir 'xyz' to the root of colab

# # Specify the path to your folder
# gdrive = '/content/drive/MyDrive/AI/2025'
# workdir = '/datasets'
# slink = '/content' + workdir
# fullpath = gdrive + workdir

# # Check if the folder exists
# if os.path.exists(fullpath):
#   # Create the symbolic link
#   try:
#     os.symlink(fullpath, slink)
#     print(f"Symbolic link created from '{fullpath}' to '{slink}'")
#   except FileExistsError:
#     print(f"Symbolic link '{slink}' already exists.")
#   except OSError as e:
#     print(f"Error creating symbolic link: {e}")
# else:
#   print(f"Error: Folder '{fullpath}' not found.")

In [ ]:
#!unzip datasets/dataset.zip -d datasets/
#!rm datasets/dataset.zip

#Notebook Start

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

folder = './datasets/crypto/hourly/'

df = pd.read_csv(folder + 'dataset.csv')
df

In [ ]:
# df_bckp = df
# df = df_bckp.iloc[17:].reset_index(drop=True)
# df

In [ ]:
date_column_name = df.columns[0]
date_format = '%Y-%m-%d' if date_column_name.lower() == 'date' else '%Y-%m-%d %H:%M:%S'

date_column_name, date_format

In [ ]:
# Converter a coluna para datetime removendo o fuso horário
if date_column_name.lower() == 'date':
    df[date_column_name] = pd.to_datetime(df[date_column_name])
else:
    df[date_column_name] = pd.to_datetime(df[date_column_name]).dt.tz_convert(None)

df

In [ ]:
# prompt: plot df using plotly
import plotly.express as px

fig = px.line(df.drop('Real', axis=1), x=date_column_name, y=df.columns[2:])
fig.show()

In [ ]:
split_date = pd.to_datetime('2024-08-26')
#split_date = pd.to_datetime('2025-01-01')

# Find the first row where the date is equal or greater than split_date
def get_split_date_index(df, split_date):
  for i in range(len(df)):
    if df.iloc[i, 0] >= split_date:
      return i

split_idx = get_split_date_index(df, split_date)
split_idx, df.iloc[split_idx, 0]

In [ ]:
split_idx += 17 # adjusting for other forecasts

In [ ]:
len(df) - split_idx

In [ ]:
# needed for dumb4cast
df_train = df.iloc[:split_idx, :].drop('Real', axis=1)
df_train

In [ ]:
# needed for dumb4cast
df_test = df.iloc[split_idx:, :].drop('Real', axis=1)
df_test

In [ ]:
# loading pre-trained model
MODEL_NAMES = [
    # 'arima',
    'lstm',
    'gru',
    'mlp',
    'dlinear',
    'nlinear',
    'informer',
    'autoformer',
    'fedformer',
    'bitcn',
    'rnn',

    'tcn',
    'deepar',
    'dilatedrnn',
    #'nbeats',
    #'nbeatsx',
    'nhits',
    'tide',
    'deepnpts',
    'tft',
    'vanilla',
    'patchtst',
    #'itransformer',
    #'timesnet'
]

In [ ]:
# prompt: add last row of df_train to a new test_df dataframe
# Por que: o backtest precisa comparar os dados reais do dia anterior à previsão pra saber se compra ou vende
real_column = df['Real'][split_idx-1:]
test_df = df.iloc[split_idx-1:, :].drop('Real', axis=1)
print(real_column)
test_df

In [ ]:
# forecasts without cross-validation
forecasts = []

forecasts_path = 'forecasts/'
path = folder + forecasts_path

for model_name in MODEL_NAMES:
  print(f'Loading {model_name.upper()} forecasts')
  forecast_df = pd.read_csv(path + 'forecast-' + model_name + '.csv')

  forecast_df = forecast_df.iloc[17:] # adjusting for cross validation forecasts

  forecasts.append(forecast_df[test_df.columns])

forecasts[0][test_df.columns].head(5)

In [ ]:
def load_crossvalidation_models(path, model_names, columns_order, debug = False, order = False):
  forecasts = []
  for model_name in model_names:
    if debug:
      print(f'Loading {model_name.upper()} forecasts')
    forecast_df = pd.read_csv(path + model_name + '.csv')
    forecast_df = forecast_df.rename(columns={'ds': 'Datetime'})
    forecast_df = forecast_df.drop(['cutoff', 'y'], axis=1)
    forecast_df = forecast_df.pivot(index='Datetime', columns='unique_id', values=forecast_df.columns[2])
    forecast_df = forecast_df.reset_index()
    if (order):
      forecast_df = forecast_df[columns_order]

    forecasts.append(forecast_df)
  if debug:
    display(forecast_df)
  return forecasts

In [ ]:
# forecasts with cross-validation (no refit)
cross_plain_path = folder + 'cross-validation/plain/'

plain_forecasts = load_crossvalidation_models(cross_plain_path + forecasts_path, MODEL_NAMES, test_df.columns, True, True)

In [ ]:
# forecasts with cross-validation (no refit)
cross_plain_btc_path = folder + 'cross-validation/plain-btc/'

plain_btc_forecasts = load_crossvalidation_models(cross_plain_btc_path + forecasts_path, MODEL_NAMES, test_df.columns, True, False)

In [ ]:
# forecasts with cross-validation and refit
cross_refit_path = folder + 'cross-validation/refit/'

refit_forecasts = load_crossvalidation_models(cross_refit_path + forecasts_path, MODEL_NAMES, test_df.columns, True, True)

In [ ]:
# creating a baseline buy-and-hold dataframe

IB = 100.0 # initial balance for each asset

# copy test_df to bh_df and set every value to zero
bh_df = test_df.copy()
for col in bh_df.columns[1:]:
  bh_df[col] = 0
bh_df[bh_df.columns[0]] = test_df[test_df.columns[0]]

# except from the first column of bh_df, divide every other first row's value by IB (buying stocks)
for j in range(1, len(test_df.columns)):
  bh_df.iloc[0, j] = IB#test_df.iloc[0, j] / IB

# calculating how much each stock would worth every day
for i in range(1, len(test_df)):
  for j in range(1, len(test_df.columns)):
    bh_df.iloc[i, j] = bh_df.iloc[i-1, j] * (test_df.iloc[i, j] / test_df.iloc[i-1, j])

bh_df.reset_index(drop=True, inplace=True)

bh_df

In [ ]:
# it looks all models prediction are just an one day shit from the real values.
# dumb_4cast is a simple one day shit from test-data, so it'll force a buy transcation if the current
# value had an increase compared to the previous day or sell if there was a drop of it's value.

dumb_4cast = df_train.tail(2).copy()
dumb_4cast = pd.concat([dumb_4cast, df_test[:-1]], ignore_index=True)
dumb_4cast.reset_index(drop=True, inplace=True)

# prompt: subtract one day for every value of column Date from dumb_4cast
# convert it first
dumb_4cast[date_column_name] = pd.to_datetime(dumb_4cast[date_column_name])

if date_column_name.lower() == 'date':
    dumb_4cast[date_column_name] = dumb_4cast[date_column_name] + pd.DateOffset(days=1)
else:
    dumb_4cast[date_column_name] = dumb_4cast[date_column_name] + pd.DateOffset(hours=1)

# prompt: change dumb_4cast Date column back to string type
dumb_4cast[date_column_name] = dumb_4cast[date_column_name].dt.strftime(date_format)

dumb_4cast

In [ ]:
MODEL_NAMES.append('dumb_4cast')
forecasts.append(dumb_4cast)

Creating a dataframe with a wallet composed by the models predictions

In [ ]:
# prompt: count how many True values are in real_column
true_count = sum(real_column)
print(f"Number of True values in real_column: {true_count}")

In [ ]:
def backtest(real_df, real_column, pred_df):
  # copy real_df to model_df and set every value to zero
  model_df = real_df.copy()
  for col in model_df.columns[1:]:
    model_df[col] = 0
  model_df[model_df.columns[0]] = real_df[real_df.columns[0]]

  model_df.reset_index(drop=True, inplace=True)

  # except from the first column of model_df, divide every other first row's value by IB (buying stocks)
  for j in range(1, len(real_df.columns)):
    model_df.iloc[0, j] = IB

  count = 0

  # calculating how much each stock would worth every day
  for i in range(1, len(real_df)):
    if not real_column.iloc[i]: # don't backtest if market is closed (filled transaction, not real)
      # copy previous row
      model_df.iloc[i, 1:] = model_df.iloc[i-1, 1:]
    else:
      count += 1
      for j in range(1, len(real_df.columns)):
        if pred_df.iloc[i-1, j] > real_df.iloc[i-1, j]: # predicted value is higher than current value: buy
          x = model_df.iloc[i-1, j] * (real_df.iloc[i, j] / real_df.iloc[i-1, j])
        else:                                           # predicted value is lower than current value: sell
          x = model_df.iloc[i-1, j] * (real_df.iloc[i-1, j] / real_df.iloc[i, j])

        if x > 0:
          model_df.iloc[i, j] = x
        else:
          model_df.iloc[i, j] = 0
          break

  print(f'\t{count} predicted steps')

  return model_df

# i = 0
# print(f'backtesting {MODEL_NAMES[i]}: ', end='')
# backtest(test_df, real_column, forecasts[i])

In [ ]:
def run_backtests(model_names, test_df, real_column, forecasts, backtests_df):
  for i in range(len(model_names)):
    print(f'backtesting {model_names[i]}: ', end='')
    bt_df = backtest(test_df, real_column, forecasts[i])
    backtests_df.append(bt_df)

  return backtests_df

In [ ]:
%%time
backtests = [bh_df]

# for i in range(len(MODEL_NAMES)):
#   print(f'backtesting {MODEL_NAMES[i]}: ', end='')
#   model_rev_df = backtest(test_df, real_column, forecasts[i])
#   backtests.append(model_rev_df)

_ = run_backtests(MODEL_NAMES, test_df, real_column, forecasts, backtests)

In [ ]:
test_df.shape, real_column.shape, forecasts[0].shape, backtests[0].shape

In [ ]:
test_df.shape, real_column.shape, plain_forecasts[0].shape

In [ ]:
MODEL_NAMES.remove('dumb_4cast')

In [ ]:
%%time
plain_backtests = []

_ = run_backtests(MODEL_NAMES, test_df, real_column, plain_forecasts, plain_backtests)

In [ ]:
%%time
plain_btc_backtests = []

_ = run_backtests(MODEL_NAMES, test_df.iloc[:, 0:2], real_column, plain_btc_forecasts, plain_btc_backtests)

In [ ]:
%%time
refit_backtests = []

_ = run_backtests(MODEL_NAMES, test_df, real_column, refit_forecasts, refit_backtests)

In [ ]:
# folder # './datasets/crypto/hourly/'
# path # folder + 'forecasts/' # forecasts without cross-validation
# cross_plain_path # folder + 'cross-validation/plain/' # forecasts with cross-validation (no refit)
# cross_refit_path # folder + 'cross-validation/refit/' # forecasts with cross-validation and refit

# df # original dataset loaded from .csv

# date_column_name # 'Datetime' or 'Date'
# date_format # '%Y-%m-%d' or '%Y-%m-%d %H:%M:%S'
# split_idx # 11616 (index where training and testing datasets are split, separated by provided date)

# df_train # df up to split_idx
# df_test # df from split_idx
# real_column # column 'Real' values extracted from df
# test_df # df from split_idx-1 dropped by column 'Real'

# MODEL_NAMES # list of all used models (appended by 'dumb_4cast' later)

# forecasts # forecasts without cross-validation
# plain_forecasts # forecasts with cross-validation (no refit)
# refit_forecasts # forecasts with cross-validation and refit

# IB # 100.0 initial balance for each asset

# bh_df # baseline buy-and-hold dataframe
# dumb_4cast # simple one day shit from df_train

# backtests # backtests calculated from forecasts
# plain_backtests # backtests calculated from plain_forecasts
# refit_backtests # backtests calculated from refit_forecasts

In [ ]:
folder, path, cross_plain_path, cross_refit_path

In [ ]:
# prompt: save all models in a 'backtest' folder inside workdir
import os

def save_backtests(folder, model_names, backtests):
  # Create the 'backtest' directory if it doesn't exist
  backtest_dir = os.path.join(folder, 'backtests')  # Use workdir variable
  os.makedirs(backtest_dir, exist_ok=True)

  # Save buy and hold and dumb4cast to the backtest folder
  # bh_filename = os.path.join(backtest_dir, "buy-and-hold.csv")
  # bh_df.to_csv(bh_filename, index=False)

  # Save each model's dataframe to a separate CSV file in the 'backtest' folder
  for i, model_name in enumerate(model_names):
      filename = os.path.join(backtest_dir, f"{model_name}.csv")
      backtests[i].to_csv(filename, index=False)  # i+1 to skip bh_df

  print(f"Backtest results saved to '{backtest_dir}'")

In [ ]:
save_backtests(cross_plain_path, MODEL_NAMES, plain_backtests)
save_backtests(cross_plain_btc_path, MODEL_NAMES, plain_btc_backtests)
save_backtests(cross_refit_path, MODEL_NAMES, refit_backtests)

In [ ]:
import IPython
IPython.display.Audio("file_example_MP3_1MG.mp3", autoplay=True)